<a href="https://colab.research.google.com/github/dimtr/PyDataEHV_workshop/blob/master/PyData_Workshop_Part1_VAE_GAN.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>



## Variational Autoencoder (VAE) ([article](https://arxiv.org/abs/1312.6114)) 

##TODO add Image -- ![variational autoencoder](imgs/vae.png)

In [2]:
### install necessary packages if in colab
def run_subprocess_command(cmd):
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    for line in process.stdout:
        print(line.decode().strip())


import sys, subprocess

IN_COLAB = "google.colab" in sys.modules
colab_requirements = [
    "pip install tf-nightly-gpu-2.0-preview==2.0.0.dev20190513",
    "pip install tfp-nightly==0.7.0.dev20190508",
]

if IN_COLAB:
    for i in colab_requirements:
        run_subprocess_command(i)

In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
%matplotlib inline
from IPython import display
import pandas as pd
import tensorflow_probability as tfp
ds = tfp.distributions

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
print(tf.__version__, tfp.__version__)